In [1]:
# Task 1: Data Analysis and Preprocessing
## Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
## Load Data
# Load the datasets
fraud_data = pd.read_csv('../data/Fraud_Data.csv')
ip_to_country = pd.read_csv('../data/IpAddress_to_Country.csv')
credit_card = pd.read_csv('../data/creditcard.csv')
## Displaying the first 5 rows of each dataset
display(fraud_data.head())
display(ip_to_country.head())
display(credit_card.head())
## Data Cleaning: Missing Values, Duplicates, and Data Types
# Check for missing values
print('Fraud_Data.csv missing values:')
display(fraud_data.isnull().sum())
print('IpAddress_to_Country.csv missing values:')
display(ip_to_country.isnull().sum())
print('creditcard.csv missing values:')
display(credit_card.isnull().sum())
# Check for duplicates
print('Fraud_Data.csv duplicates:', fraud_data.duplicated().sum())
print('IpAddress_to_Country.csv duplicates:', ip_to_country.duplicated().sum())
print('creditcard.csv duplicates:', credit_card.duplicated().sum())
# Remove duplicates
fraud_data = fraud_data.drop_duplicates()
ip_to_country = ip_to_country.drop_duplicates()
credit_card = credit_card.drop_duplicates()
# Check and correct data types
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])
ip_to_country['lower_bound_ip_address'] = pd.to_numeric(ip_to_country['lower_bound_ip_address'], errors='coerce')
ip_to_country['upper_bound_ip_address'] = pd.to_numeric(ip_to_country['upper_bound_ip_address'], errors='coerce')
### Data cleaning complete. Ready for EDA.



,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Fraud_Data.csv missing values:


user_id           0
signup_time       0
purchase_time     0
purchase_value    0
device_id         0
source            0
browser           0
sex               0
age               0
ip_address        0
class             0
dtype: int64

IpAddress_to_Country.csv missing values:


lower_bound_ip_address    0
upper_bound_ip_address    0
country                   0
dtype: int64

creditcard.csv missing values:


Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

Fraud_Data.csv duplicates: 0
IpAddress_to_Country.csv duplicates: 0
creditcard.csv duplicates: 1081


In [2]:
### Merge Fraud Data with IP-to-Country Mapping
# --- Geolocation Merge: Map IP Address to Country ---
# Ensure columns are numeric and sorted
fraud_data['ip_address'] = pd.to_numeric(fraud_data['ip_address'], errors='coerce')
ip_to_country['lower_bound_ip_address'] = pd.to_numeric(ip_to_country['lower_bound_ip_address'], errors='coerce')
ip_to_country['upper_bound_ip_address'] = pd.to_numeric(ip_to_country['upper_bound_ip_address'], errors='coerce')

# Sort both DataFrames for merge_asof
fraud_data_sorted = fraud_data.sort_values('ip_address').reset_index(drop=True)
ip_to_country_sorted = ip_to_country.sort_values('lower_bound_ip_address').reset_index(drop=True)

# Use merge_asof to find the lower bound
merged = pd.merge_asof(
    fraud_data_sorted,
    ip_to_country_sorted,
    left_on='ip_address',
    right_on='lower_bound_ip_address',
    direction='backward'
)

# Now filter to only those where ip_address <= upper_bound_ip_address
merged['country'] = np.where(
    merged['ip_address'] <= merged['upper_bound_ip_address'],
    merged['country'],
    'Unknown'
)

# If you want to keep the original order:
merged = merged.sort_index()

# Check the result
merged[['ip_address', 'country']].head()

,ip_address,country
0,52093.496895,Unknown
1,93447.138961,Unknown
2,105818.501505,Unknown
3,117566.664867,Unknown
4,131423.789042,Unknown


In [3]:
### Merge Fraud Data with IP-to-Country Mapping
# --- Geolocation Merge: Map IP Address to Country ---
# Ensure columns are numeric and sorted
fraud_data['ip_address'] = pd.to_numeric(fraud_data['ip_address'], errors='coerce')
ip_to_country['lower_bound_ip_address'] = pd.to_numeric(ip_to_country['lower_bound_ip_address'], errors='coerce')
ip_to_country['upper_bound_ip_address'] = pd.to_numeric(ip_to_country['upper_bound_ip_address'], errors='coerce')

# Sort both DataFrames for merge_asof
fraud_data_sorted = fraud_data.sort_values('ip_address').reset_index(drop=True)
ip_to_country_sorted = ip_to_country.sort_values('lower_bound_ip_address').reset_index(drop=True)

# Use merge_asof to find the lower bound
merged = pd.merge_asof(
    fraud_data_sorted,
    ip_to_country_sorted,
    left_on='ip_address',
    right_on='lower_bound_ip_address',
    direction='backward'
)

# Now filter to only those where ip_address <= upper_bound_ip_address
merged['country'] = np.where(
    merged['ip_address'] <= merged['upper_bound_ip_address'],
    merged['country'],
    'Unknown'
)

# If you want to keep the original order:
merged = merged.sort_index()

# Check the result
merged[['ip_address', 'country']].head()

,ip_address,country
0,52093.496895,Unknown
1,93447.138961,Unknown
2,105818.501505,Unknown
3,117566.664867,Unknown
4,131423.789042,Unknown


In [4]:
   # Number of transactions per user
   merged['user_transaction_count'] = merged.groupby('user_id')['user_id'].transform('count')

   # Preview the result
   merged[['user_id', 'user_transaction_count']].head()

,user_id,user_transaction_count
0,62421,1
1,173212,1
2,242286,1
3,370003,1
4,119824,1


In [5]:
# Number of transactions per device
merged['device_transaction_count'] = merged.groupby('device_id')['device_id'].transform('count')

# Preview the result
merged[['device_id', 'device_transaction_count']].head()

,device_id,device_transaction_count
0,ZCLZTAJPCRAQX,1
1,YFGYOALADBHLT,1
2,QZNVQTUITFTHH,1
3,PIBUQMBIELMMG,1
4,WFIIFCPIOGMHT,1


In [6]:
# Number of transactions per IP address
merged['ip_transaction_count'] = merged.groupby('ip_address')['ip_address'].transform('count')

# Preview the result
merged[['ip_address', 'ip_transaction_count']].head()

,ip_address,ip_transaction_count
0,52093.496895,1
1,93447.138961,1
2,105818.501505,1
3,117566.664867,1
4,131423.789042,1


In [7]:
# Number of transactions per country
merged['country_transaction_count'] = merged.groupby('country')['country'].transform('count')

# Preview the result
merged[['country', 'country_transaction_count']].head()

,country,country_transaction_count
0,Unknown,21966
1,Unknown,21966
2,Unknown,21966
3,Unknown,21966
4,Unknown,21966


In [8]:
# Sort by user and purchase time
merged = merged.sort_values(['user_id', 'purchase_time'])

# Calculate time since previous transaction for each user (in hours)
merged['time_since_prev_txn_user'] = merged.groupby('user_id')['purchase_time'].diff().dt.total_seconds() / 3600

# Preview the result
merged[['user_id', 'purchase_time', 'time_since_prev_txn_user']].head(10)

,user_id,purchase_time,time_since_prev_txn_user
31545,2,2015-02-21 10:03:37,NaN
97542,4,2015-09-26 21:32:16,NaN
12873,8,2015-08-13 11:53:07,NaN
27671,9,2015-05-20 23:06:42,NaN
104500,12,2015-03-04 20:56:37,NaN
21011,16,2015-03-12 12:46:23,NaN
127701,18,2015-10-23 00:18:57,NaN
149684,26,2015-03-21 09:04:08,NaN
42090,33,2015-10-28 18:12:41,NaN
109634,39,2015-01-08 18:13:26,NaN


In [9]:
# Sort by device and purchase time
merged = merged.sort_values(['device_id', 'purchase_time'])

# Calculate time since previous transaction for each device (in hours)
merged['time_since_prev_txn_device'] = merged.groupby('device_id')['purchase_time'].diff().dt.total_seconds() / 3600

# Preview the result
merged[['device_id', 'purchase_time', 'time_since_prev_txn_device']].head(10)

,device_id,purchase_time,time_since_prev_txn_device
87854,AAALBGNHHVMKG,2015-06-13 23:42:18,NaN
29042,AAAWIHVCQELTP,2015-03-29 00:39:07,NaN
114,AAAXJHWCLISKY,2015-07-22 03:18:01,NaN
48516,AAAXXOZJRZRAO,2015-01-01 04:25:23,NaN
48518,AAAXXOZJRZRAO,2015-01-01 04:25:24,0.000278
48517,AAAXXOZJRZRAO,2015-01-01 04:25:25,0.000278
48515,AAAXXOZJRZRAO,2015-01-01 04:25:26,0.000278
48512,AAAXXOZJRZRAO,2015-01-01 04:25:27,0.000278
48511,AAAXXOZJRZRAO,2015-01-01 04:25:28,0.000278
48513,AAAXXOZJRZRAO,2015-01-01 04:25:29,0.000278


In [10]:
# Sort by IP address and purchase time
merged = merged.sort_values(['ip_address', 'purchase_time'])

# Calculate time since previous transaction for each IP address (in hours)
merged['time_since_prev_txn_ip'] = merged.groupby('ip_address')['purchase_time'].diff().dt.total_seconds() / 3600

# Preview the result
merged[['ip_address', 'purchase_time', 'time_since_prev_txn_ip']].head(10)

,ip_address,purchase_time,time_since_prev_txn_ip
0,52093.496895,2015-03-08 10:00:39,NaN
1,93447.138961,2015-03-20 17:23:45,NaN
2,105818.501505,2015-05-26 08:54:34,NaN
3,117566.664867,2015-05-28 21:09:13,NaN
4,131423.789042,2015-04-05 07:31:46,NaN
5,155130.397004,2015-06-27 04:35:25,NaN
6,155399.107925,2015-08-14 22:21:41,NaN
7,159119.791297,2015-08-04 04:43:58,NaN
8,166326.775242,2015-03-21 09:09:10,NaN
9,207999.643852,2015-05-20 20:15:35,NaN


In [11]:
!pip install imbalanced-learn

In [12]:
from imblearn.over_sampling import SMOTE

In [13]:
     import sys
     print(sys.executable)

C:\Users\Asiya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


In [14]:
     !{sys.executable} -m pip install imbalanced-learn

In [15]:
from imblearn.over_sampling import SMOTE

In [ ]:
# Check for NaN or infinite values
print("NaN values:\n", X_train_numeric.isnull().sum())
print("Infinite values:\n", np.isinf(X_train_numeric).sum())

# Fill NaN with 0 (or use another strategy if you prefer)
X_train_numeric = X_train_numeric.fillna(0)
X_test_numeric = X_test_numeric.fillna(0)

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_numeric, y_train)

# Check new class distribution
print(pd.Series(y_train_resampled).value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Example: select features and target
features = [col for col in merged.columns if col not in ['class', 'purchase_time', 'signup_time']]
X = merged[features]
y = merged['class']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check new class distribution
print(pd.Series(y_train_resampled).value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Select only numeric columns for SMOTE
numeric_features = merged.select_dtypes(include=[np.number]).columns.tolist()
# Remove the target and any time columns you don't want as features
for col in ['class', 'purchase_time', 'signup_time']:
    if col in numeric_features:
        numeric_features.remove(col)

X = merged[numeric_features]
y = merged['class']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Fill any NaN values (if present)
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check new class distribution
print(pd.Series(y_train_resampled).value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Select only numeric columns for SMOTE
numeric_features = merged.select_dtypes(include=[np.number]).columns.tolist()
# Remove the target and any time columns you don't want as features
for col in ['class', 'purchase_time', 'signup_time']:
    if col in numeric_features:
        numeric_features.remove(col)

X = merged[numeric_features]
y = merged['class']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Fill any NaN values (if present)
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Check new class distribution
print(pd.Series(y_train_resampled).value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Define your categorical columns
categorical_cols = ['device_id', 'source', 'browser', 'sex', 'country']

# Define your numeric columns (excluding target and time columns)
numeric_cols = merged.select_dtypes(include=[np.number]).columns.tolist()
for col in ['class', 'purchase_time', 'signup_time']:
    if col in numeric_cols:
        numeric_cols.remove(col)

# Prepare X and y
X = pd.concat([merged[numeric_cols], pd.get_dummies(merged[categorical_cols], drop_first=True)], axis=1)
y = merged['class']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE

# Only use numeric columns for SMOTE
X_train_numeric = X_train[numeric_cols].fillna(0)

smote = SMOTE(random_state=42)
X_train_resampled_numeric, y_train_resampled = smote.fit_resample(X_train_numeric, y_train)

# Get the indices of the resampled data to align categorical features
resampled_indices = X_train_numeric.index[smote.sample_indices_]
X_train_resampled_categorical = X_train.loc[resampled_indices].drop(columns=numeric_cols)

In [ ]:
from imblearn.over_sampling import SMOTE

# Only use numeric columns for SMOTE
X_train_numeric = X_train[numeric_cols].fillna(0)

smote = SMOTE(random_state=42)
X_train_resampled_numeric, y_train_resampled = smote.fit_resample(X_train_numeric, y_train)

# Get the indices of the resampled data to align categorical features
resampled_indices = X_train_numeric.index[smote.sample_indices_]
X_train_resampled_categorical = X_train.loc[resampled_indices].drop(columns=numeric_cols)

In [ ]:
# Combine numeric and categorical features after SMOTE
X_train_resampled = pd.concat([pd.DataFrame(X_train_resampled_numeric, columns=numeric_cols).reset_index(drop=True),
                               X_train_resampled_categorical.reset_index(drop=True)], axis=1)

In [ ]:
scaler = StandardScaler()
X_train_resampled[numeric_cols] = scaler.fit_transform(X_train_resampled[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [ ]:
scaler = StandardScaler()
X_train_resampled[numeric_cols] = scaler.fit_transform(X_train_resampled[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])